# Creating the initial mesh
Made on 12/02/2026

Kernel: remeshing (Python 3.13.9)

Relevant code from `init_mesh.ipynb` that creates a mesh in either HEX8 or HEX20 and saves it to a meshes folder. 

Libraries, functions and so forth.

In [ ]:
# importing the relevant packages
import pyvista as pv
import scipy.spatial as sp
import meshio
import os
import matplotlib.pyplot as plt
import numpy as np
import pygmsh as pg
import jax.numpy as jnp
import gmsh
from units_constants import ConversionFactors as uc

# get meshio cell type
def get_meshio_cell_type(ele_type):
    """Convert element type into a compatible string with 
    `meshio <https://github.com/nschloe/meshio/blob/9dc6b0b05c9606cad73ef11b8b7785dd9b9ea325/src/meshio/xdmf/common.py#L36>`_.

    Parameters
    ----------
    ele_type : str
        :attr:`~jax_fem.fe.FiniteElement.ele_type`

    Returns
    -------
    cell_type : str
        Compatible with meshio.
    """
    if ele_type == 'TET4':
        cell_type = 'tetra'
    elif ele_type == 'TET10':
        cell_type = 'tetra10'
    elif ele_type == 'HEX8':
        cell_type = 'hexahedron'
    elif  ele_type == 'HEX20':
        cell_type = 'hexahedron20'
    else:
        raise NotImplementedError
    return cell_type


# adapted from jax-fem source code, https://github.com/deepmodeling/jax-fem/blob/main/jax_fem/generate_mesh.py#L151
def box_mesh_gmsh(Nx, Ny, Nz, domain_x, domain_y, domain_z, data_dir=None, ele_type='HEX8'):
    """
    My additions:
        * works with HEX20 too
        * degree is hard-coded
        * using the newest gmsh file format (jax-FEM used 2.2, I use 4.1)
        * saves to "$HOME/Remeshing/Fluffy-remesh/Meshes" if no other data_dir is specified
    
    ---
    Jax-fem:
    Generate a box mesh with the help of `gmsh <https://gmsh.info/>`_.
    Some useful links include
    `tutorial_hex <https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/examples/api/hex.py>`_, 
    `tutorial_t1 <https://gitlab.onelab.info/gmsh/gmsh/-/blob/gmsh_4_7_1/tutorial/python/t1.py>`_, 
    `tutorial_t3 <https://gitlab.onelab.info/gmsh/gmsh/-/blob/gmsh_4_7_1/tutorial/python/t3.py>`_.

    04/01/2026: currently only accepts hex cells.
    Parameters
    ----------
    Nx : int
        Number of nodes along x-axis.
    Ny : int
        Number of nodes along y-axis.
    Nz : int
        Number of nodes along z-axis.
    domain_x : float
        Length of side along x-axis.
    domain_y : float
        Length of side along y-axis.
    domain_z : float
        Length of side along z-axis.    
    data_dir : str, optional
        A directory to store the generated mesh.
    ele_type : str
        The type is :attr:`~jax_fem.fe.FiniteElement.ele_type`.
        Accept 'HEX8', 'HEX20'
    cell_type : str
        The compatible string from the ele type for meshio. 
        See https://github.com/nschloe/meshio/blob/9dc6b0b05c9606cad73ef11b8b7785dd9b9ea325/src/meshio/xdmf/common.py#L36%3E
    degree : int
        Degree of the element.

    Returns
    -------
    out_mesh : MeshioMesh
        Mesh in meshio format.
    """

    # assert ele_type != 'HEX20', f"gmsh cannot produce HEX20 mesh?"

    # cell_type = get_meshio_cell_type(ele_type)
    """ get_elements uses the basix library (of FEniCs?) to retrieve a bunch of element info, of which just
    the degree is used in this particular function. So at least for now, no need to copy that code. 
    """
    # the degrees below are directly copied from the get_elements function, so it should work
    if ele_type == 'HEX8':
        degree = 1
    elif ele_type == "HEX20":
        degree = 2
    elif ele_type == "HEX27":
        degree = 2

    # setting up the datafile
    #_, _, _, _, degree, _ = get_elements(ele_type) 
    if data_dir:
        msh_dir = os.path.join(data_dir, 'Meshes')
    else: 
        msh_dir = "$HOME/Remeshing/Fluffy-remesh/Meshes"
    os.makedirs(msh_dir, exist_ok=True)
    msh_file = os.path.join(msh_dir, f'box{ele_type}.msh')

 
    offset_x = 0.
    offset_y = 0.
    offset_z = 0.

    gmsh.initialize()
    # gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)  # save in old MSH format
    gmsh.option.setNumber("Mesh.MshFileVersion", 4.1)  # what if I use a newer version? Update: nope, doesn't matter

    cell_type = get_meshio_cell_type(ele_type)
    if cell_type.startswith('tetra'):
        Rec2d = False  # tris or quads
        Rec3d = False  # tets, prisms or hexas
    else:
        Rec2d = True
        Rec3d = True

    p = gmsh.model.geo.addPoint(offset_x, offset_y, offset_z)
    l = gmsh.model.geo.extrude([(0, p)], domain_x, 0, 0, [Nx], [1])
    s = gmsh.model.geo.extrude([l[1]], 0, domain_y, 0, [Ny], [1], recombine=Rec2d)
    v = gmsh.model.geo.extrude([s[1]], 0, 0, domain_z, [Nz], [1], recombine=Rec3d)
    gmsh.option.setNumber('Mesh.SecondOrderIncomplete', 1) # needed for HEX20 to actually produce HEX20 and ot HEX27


    # something
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(3) # mesh dims
    gmsh.model.mesh.setOrder(degree) # set element order to degree
    gmsh.write(msh_file)
    gmsh.finalize() 

    # save gmsh mesh into meshio format, although I am *still* not sure which format that is, or why that's necessary
    mesh = meshio.read(msh_file)
    points = mesh.points # (num_total_nodes, dim)
    cells =  mesh.cells_dict[cell_type] # (num_cells, num_nodes)
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})

    return mesh, out_mesh

## Running the code

In [ ]:
# path to mesh folder
mpath = "./Meshes/" # need to change this so that it also leads to data_dir if specified

mesh_HEX8, out_mesh_HEX8 = box_mesh_gmsh(4,4,4,1,1,1, ele_type='HEX8')
#mesh_HEX20, out_mesh_HEX20 = box_mesh_gmsh(4,4,4,1,1,1, ele_type='HEX20')
#print(mesh_HEX8)

# plotting
boxmesh = pv.read(mpath+"boxHEX8.msh")
#boxmesh

# matplotlib plot of points
Xs = boxmesh.points[:,0]
Ys = boxmesh.points[:,1]
Zs = boxmesh.points[:,2]
x_max = Xs.max()
y_max=Ys.max()
z_max = Zs.max()


# labelled points
ax = plt.figure(dpi = 200).add_subplot(projection='3d')
for i in range (len(boxmesh.points)):
    ax.text(boxmesh.points[i,0], boxmesh.points[i,1], boxmesh.points[i,2], str(i), color='red', fontsize=6)
ax.text(.4*x_max, .2*y_max, 1.4*z_max, "Order of mesh points", color='k', fontsize=12)
ax.scatter(Xs, Ys, Zs)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=20., azim=-35, roll=0)
ax.set_aspect('equal') # make the grid the same shape as the actual object
plt.show()